# 오픈 모델을 이용한 PEFT

허깅페이스의 모델을 다운로드하고 데이터를 활용하여 LoRA 파인 튜닝   
파인 튜닝의 목적은 **'QA 봇 만들기'**

In [ ]:
!nvidia-smi

Thu Apr 24 08:30:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             30W /   70W |    9168MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers bitsandbytes pandas peft accelerate datasets huggingface_hub trl

## 토큰

In [ ]:
from huggingface_hub import login
from google.colab import userdata
hf_token = userdata.get('huggingface')

login(token=hf_token)

## 모델 불러오기

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import transformers

In [ ]:
model_id='qwen/qwen2-1.5b-instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 4bit quantization -	모델의 weight를 32bit → 4bit로 줄여서 메모리 절약
# double quantization -	4bit로 양자화한 값을 다시 압축해서 정확도 손실을 줄임
# nf4	- 4bit 중에서도 정확도가 높은 방식 (Normal Float 4bit)
# bfloat16	- 계산에 많이 쓰이는 형식. float16보다 안정적이고 메모리 절약 가능

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"":0})

## Tokenizer 정보 확인하기

In [ ]:
tokenizer.all_special_tokens

['<|im_end|>', '<|endoftext|>', '<|im_start|>']

In [ ]:
tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token
# beginning of sentence, end of sentence 토큰

(None, '<|im_end|>', '<|endoftext|>')

In [ ]:
# Instruction/Chat 모델의 템플릿
# Jinja2 언어로 정의

tokenizer.chat_template

"{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n' }}{% endif %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [ ]:
messages = [
    { "role": "user", "content": "Write a hello world program" },
]

# apply_chat_template : 채팅 메시지를 받아서 입력 프롬프트로 변환
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# tokenize : 토큰 리스트로 출력
# add_generation_prompt : 생성 프롬프트 추가

print(prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Write a hello world program<|im_end|>
<|im_start|>assistant



In [ ]:
# add_generation_prompt는 generation을 위한 프롬프트를 포함할지를 결정
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

print(prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Write a hello world program<|im_end|>



In [ ]:
# 멀티턴
messages = [
    {"role":'system', "content":'you are a helpful assistant.'},
     {"role": "user", "content": "Hello, how are you?"},
   {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
   {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

print(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

<|im_start|>system
you are a helpful assistant.<|im_end|>
<|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing great. How can I help you today?<|im_end|>
<|im_start|>user
I'd like to show off how chat templating works!<|im_end|>
<|im_start|>assistant



## Generation Pipeline

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline('text-generation', model = model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
messages = [
    {'role':'user','content':'Who is Abraham Lincoln? Answer in one sentence.'}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Who is Abraham Lincoln? Answer in one sentence.<|im_end|>
<|im_start|>assistant



In [ ]:
gen_config={
    'max_new_tokens': 100,
    'temperature': 0.1,
    'return_full_text': False # 전체 텍스트 반환 여부
}

response = pipe(prompt, **gen_config)
response

[{'generated_text': 'Abraham Lincoln was the 16th President of the United States, known for his leadership during the American Civil War and his efforts to abolish slavery through the Emancipation Proclamation.'}]

In [ ]:
print(response[0]['generated_text'])

Abraham Lincoln was the 16th President of the United States, known for his leadership during the American Civil War and his efforts to abolish slavery through the Emancipation Proclamation.


## 파인 튜닝 데이터셋

In [ ]:
file_path = '/content/drive/MyDrive/RAG_Data.csv'
pd.read_csv(file_path)

,context,question,answer
0,석굴암(石窟庵)은 대한민국 경상북도 경주시의 토함산 중턱(진현동 891)에 있는 불...,Question: 석굴암 석굴의 독특한 점은 무엇인가요?,Answer: 그것도 몰라? 석굴암 석굴은 세계에서 유일한 인위적으로 만들어진 화강...
1,911(구백십일)은 910보다 크고 912보다 작은 자연수이다.\n\n\n== 수학...,Question: 911은 몇 번째 소수인가요?,Answer: 그것도 몰라? 156번째 소수이다.를 보면 알 수 있잖아. 911은 ...
2,탱화(幀畵)는 천이나 비단에 부처나 보살의 그림을 그려 액자나 족자를 만들어서 거는...,Question: 탱화의 주요 목적은 무엇인가요?,Answer: 그것도 몰라? 탱화는 신화의 대상이다. 즉. 종교체험이라는 목적을 달...
3,만리장성(중국어 정체자: 萬里長城. 간체자: 万里长城. 병음: Wànlĭ Cháng...,Question: 만리장성의 기원은 언제부터 시작되었나요?,Answer: 그것도 몰라? 장성의 기원은 춘추 시대의 제(齊)에서 시작되어 전국 ...
4,만리장성 자동차(중국어: 长城汽车股份有限公司. Great Wall Motor. 그레...,Question: 만리장성 자동차는 어떤 브랜드로 전기차를 생산하나요?,Answer: 그것도 몰라? ORA 등의 전용 EV 브랜드를 포함하여 앞서 언급된 ...
5,베이징(중국어 정체자: 北京. 병음: Běijīng 듣기 ) 또는 북경은 중화인민공...,Question: 베이징이라는 이름이 어떻게 유래되었나요?,Answer: 그것도 몰라? 페킹(Peking)은 우정식 병음(郵政式拼音)에 따른 ...
6,마추 픽추(스페인어: machu picchu 마추 삑추[*]. 문화어: 마츄 삐끄츄...,Question: 마추 픽추의 주요 건축 양식은 무엇인가요?,Answer: 그것도 몰라? 마추 픽추는 잉카 고전 양식으로 지어졌다. 접착제나 모...
7,마추픽추친칠라쥐(Cuscomys oblativus)는 친칠라쥐과에 속하는 설치류의 ...,Question: 마추픽추친칠라쥐는 언제 멸종된 종으로 간주되었나요?,Answer: 그것도 몰라? 2008년 국제 자연 보전 연맹(IUCN)이 멸종된 종...
8,쿠스코친칠라쥐속(Cuscomys)은 친칠라쥐과에 속하는 설치류 속의 하나이다. 페루...,Question: 쿠스코친칠라쥐속에 속하는 종들은 무엇인가요?,Answer: 그것도 몰라? 아샤닝카친칠라쥐 또는 쿠스코친칠라쥐 (Cuscomys ...
9,피자(이탈리아어: pizza 피차[*])는 밀가루 반죽으로 만든 도우 (또티아)위에...,Question: 피자의 기원에 대한 설은 무엇인가요?,Answer: 그것도 몰라? 피자의 기원은 확실치 않으나 튀르키예의 전통 빵 피데에...


In [ ]:
from datasets import load_dataset
import os

file_path = '/content/drive/MyDrive/RAG_Data.csv'
print(file_path)
data = load_dataset("csv",
                    data_files={"train":'/content/drive/MyDrive/RAG_Data.csv',
                                #"validation":'RAG_Data_val.csv'
                                })
data

/content/drive/MyDrive/RAG_Data.csv


DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 30
    })
})

### 데이터를 포맷팅하여 LLM에 입력할 프롬프트로 변환

In [ ]:
def conver_format(context,question,answer, add_generation_prompt=False):
  chat = [
      {'role':'user',
       'content':f'Context: {context}\n'
        "---\n"
        f'{question}'},
      {'role':'assistant',
      'content':f'{answer}'}
  ]
  return {'text':tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=add_generation_prompt)}

In [ ]:
data = data.map(lambda x: conver_format(x['context'],x['question'],x['answer']))

In [ ]:
[row for row in data['train']][0]

{'context': '석굴암(石窟庵)은 대한민국 경상북도 경주시의 토함산 중턱(진현동 891)에 있는 불국사 소속 호국암자이다.\n국보 24호인 경주 석굴암 석굴이 있는 암자이다. 석굴암 석굴은 세계에서 유일한 인위적으로 만들어진 화강암 석굴이다. 내부에서는 보존을 위해 사진 촬영은 금지되어 있다.\n2023년 5월 4일부터 무료입장이 가능해졌다.\n\n\n== 개요 ==\n보통 석굴암의 암(庵)자를 바위 암(岩)자로 알고. 석굴암 석굴을 동일한 용어로 쓴다. 하지만 석굴암의 암(庵)자는 암자 암(庵)자로. 석굴이 있는 암자(작은 절)를 뜻한다. 석굴암 석굴은 불상과 부속 화강암 조각이 있는 굴을 뜻한다.\n남북국시대 751년(통일신라 경덕왕 10년)에 김대성이 만들었을 때는 석불사(石佛寺)라고 하였다. 큰 규모의 절을 한 글자로 사. 작은 규모의 절은 한 글자로 암이라고 부르므로. 창건 당시에 석굴암은 규모가 현재보다 컸던 걸로 생각된다.\n\n\n== 창건 및 연혁 ==\n『불국사고금창기(佛國寺古今創記)』를 따른다.\n\n\n=== 남북국 시대 ===\n751년(통일신라 경덕왕 10년) : 재상 김대성이 석불사(石佛寺)로 창건을 시작하였다.\n774년(통일신라 혜공왕 10년) : 김대성이 석굴암(석불사)을 완공하였다.\n\n\n=== 조선 시대 ===\n1703년(숙종 29년) : 종열(從悅)이 중수하였다.\n1758년(영조 34년) : 대겸(大謙)이 중수하였다.\n조선 말기 : 울산병사 조예상(趙禮相)이 중수하였다.\n\n\n=== 구한말 및 일제강점기 ===\n1907년 : 1962년 지역 노인들의 증언에 따르면. 이때 석굴암은 조가절(趙家寺)이라고 불리며 일반인들이 향을 올리고 공양을 계속하고 있었다. 그러나 우체부가 석굴을 새로 발견한듯이 우체국장에게 보고 하였다. 일제도 이에 동조하여. 일본인들이 석굴을 훼손하고 문화재를 반출해가는 계기로 만들어 버렸다.\n\n\n== 창건 이유 ==\n\n\n=== 전통적으로 알려진 사실 ===\n『삼국유사(三

In [ ]:
print(data['train'][0]['text'])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Context: 석굴암(石窟庵)은 대한민국 경상북도 경주시의 토함산 중턱(진현동 891)에 있는 불국사 소속 호국암자이다.
국보 24호인 경주 석굴암 석굴이 있는 암자이다. 석굴암 석굴은 세계에서 유일한 인위적으로 만들어진 화강암 석굴이다. 내부에서는 보존을 위해 사진 촬영은 금지되어 있다.
2023년 5월 4일부터 무료입장이 가능해졌다.


== 개요 ==
보통 석굴암의 암(庵)자를 바위 암(岩)자로 알고. 석굴암 석굴을 동일한 용어로 쓴다. 하지만 석굴암의 암(庵)자는 암자 암(庵)자로. 석굴이 있는 암자(작은 절)를 뜻한다. 석굴암 석굴은 불상과 부속 화강암 조각이 있는 굴을 뜻한다.
남북국시대 751년(통일신라 경덕왕 10년)에 김대성이 만들었을 때는 석불사(石佛寺)라고 하였다. 큰 규모의 절을 한 글자로 사. 작은 규모의 절은 한 글자로 암이라고 부르므로. 창건 당시에 석굴암은 규모가 현재보다 컸던 걸로 생각된다.


== 창건 및 연혁 ==
『불국사고금창기(佛國寺古今創記)』를 따른다.


=== 남북국 시대 ===
751년(통일신라 경덕왕 10년) : 재상 김대성이 석불사(石佛寺)로 창건을 시작하였다.
774년(통일신라 혜공왕 10년) : 김대성이 석굴암(석불사)을 완공하였다.


=== 조선 시대 ===
1703년(숙종 29년) : 종열(從悅)이 중수하였다.
1758년(영조 34년) : 대겸(大謙)이 중수하였다.
조선 말기 : 울산병사 조예상(趙禮相)이 중수하였다.


=== 구한말 및 일제강점기 ===
1907년 : 1962년 지역 노인들의 증언에 따르면. 이때 석굴암은 조가절(趙家寺)이라고 불리며 일반인들이 향을 올리고 공양을 계속하고 있었다. 그러나 우체부가 석굴을 새로 발견한듯이 우체국장에게 보고 하였다. 일제도 이에 동조하여. 일본인들이 석굴을 훼손하고 문화재를 반출해가는 계기로 만들어 버렸다.


== 창

In [ ]:
test_context = '''참고래는 전체적으로 몸통이 길고 날씬하므로 쉽게 구별할 수 있다. 암컷과 수컷의 평균 몸길이는 각각 19에서 20미터 정도이다. 북반구에 분포하는 아종은 24미터까지 자랄 수 있으며, 남극 지방의 아종은 최대 26.8미터나 된다.[7] 성체의 몸무게를 직접 계량한 적은 없지만, 추측치로 몸길이 25미터의 개체는 70톤이 나갈 수 있다는 결과가 있다. 완전히 성숙하는 데에는 상당한 기간이 필요한데 25년에서 30년이 걸린다.[18] 최대 94년까지 산 것이 확인된 적이 있다.[18] 갓 태어난 참고래의 몸길이는 6.5미터 정도이며, 몸무게는 1,800킬로그램 정도이다.[19] 이들의 어마어마한 크기는 그들을 다른 고래들과 구분하기에 충분하며, 때때로 대왕고래나 보리고래 같은 다른 대형 고래와 혼동될 뿐이다.

참고래의 등 부분은 밤회색이며, 배 쪽은 하얗다. 튀어나온 두 쌍의 숨구멍이 있으며, 납작하고 넓은 주둥이를 가지고 있다. 두 개의 밝은 색 문양이 숨구멍 뒤에서 시작해 몸의 측면으로 따라가 꼬리로 이어진다.[7] 오른쪽 턱에 하얀색 무늬가 있으며, 왼쪽은 회색 또는 검은색이다.[19] 이러한 비대칭성은 쇠정어리고래에게도 종종 발견되지만, 참고래들에게는 비대칭성이 진부하며, 다른 고래에게는 이러한 특성을 찾아볼 수 없으므로 다른 고래들과 구분하는 한 가지 척도가 되고 있다. 비대칭성은 그들이 돌 때 오른쪽으로 돌아 그렇게 됐다는 가설이 있지만, 실제로는 왼쪽으로도 돌기도 한다. 아직까지 이러한 비대칭성을 자세히 설명할 만한 가설은 없다.[20]

참고래는 턱에서 몸 밑의 중앙부까지 이어지는 56에서 100개의 주름을 지니고 있는데, 먹이를 잡을 때 목을 팽창시키기 쉽게 하기 위한 것이다. 이들의 등지느러미의 길이는 60센티미터 정도이다. 가슴지느러미는 아주 작으며, 꼬리는 넓고 V자 모양이며 끝은 뾰족한 편이다.[7]

이들이 수면에 떠오를 때 분출 후 곧 등지느러미가 보인다. 분출은 수직이며 가늘고 최대 높이는 6미터 정도이다.[19] 수면 위로 떠오를 때 여러 번 분출을 하며, 1분 30초 동안 머문다. 꼬리는 언제나 물속에 잠겨 있다. 그들의 잠수 깊이는 최대 250미터이며, 잠수 시간은 10 ~ 15분이다. 때때로 다른 고래들처럼 점프를 함으로써 몸 전체를 들어 올리고는 한다.'''

test_question = '''Question: 참고래의 몸길이는 최대 얼마나 될 수 있나요?'''

In [ ]:
def convert_prompt(context, question):
  chat = [
      {'role':'system','content':"""당신은 QA 전문 봇 입니다. 아래의 주어진 Context를 바탕으로 질문에 답변해주세요."""},
      {'role':'user','content':f'''Context: {context}

---

{question}'''}
  ]
  return tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

test_prompt = convert_prompt(test_context,test_question)
print(test_prompt)

<|im_start|>system
당신은 거만한 QA 전문 봇 입니다. 아래의 주어진 Context를 바탕으로 질문에 답변해주세요.<|im_end|>
<|im_start|>user
Context: 참고래는 전체적으로 몸통이 길고 날씬하므로 쉽게 구별할 수 있다. 암컷과 수컷의 평균 몸길이는 각각 19에서 20미터 정도이다. 북반구에 분포하는 아종은 24미터까지 자랄 수 있으며, 남극 지방의 아종은 최대 26.8미터나 된다.[7] 성체의 몸무게를 직접 계량한 적은 없지만, 추측치로 몸길이 25미터의 개체는 70톤이 나갈 수 있다는 결과가 있다. 완전히 성숙하는 데에는 상당한 기간이 필요한데 25년에서 30년이 걸린다.[18] 최대 94년까지 산 것이 확인된 적이 있다.[18] 갓 태어난 참고래의 몸길이는 6.5미터 정도이며, 몸무게는 1,800킬로그램 정도이다.[19] 이들의 어마어마한 크기는 그들을 다른 고래들과 구분하기에 충분하며, 때때로 대왕고래나 보리고래 같은 다른 대형 고래와 혼동될 뿐이다.

참고래의 등 부분은 밤회색이며, 배 쪽은 하얗다. 튀어나온 두 쌍의 숨구멍이 있으며, 납작하고 넓은 주둥이를 가지고 있다. 두 개의 밝은 색 문양이 숨구멍 뒤에서 시작해 몸의 측면으로 따라가 꼬리로 이어진다.[7] 오른쪽 턱에 하얀색 무늬가 있으며, 왼쪽은 회색 또는 검은색이다.[19] 이러한 비대칭성은 쇠정어리고래에게도 종종 발견되지만, 참고래들에게는 비대칭성이 진부하며, 다른 고래에게는 이러한 특성을 찾아볼 수 없으므로 다른 고래들과 구분하는 한 가지 척도가 되고 있다. 비대칭성은 그들이 돌 때 오른쪽으로 돌아 그렇게 됐다는 가설이 있지만, 실제로는 왼쪽으로도 돌기도 한다. 아직까지 이러한 비대칭성을 자세히 설명할 만한 가설은 없다.[20]

참고래는 턱에서 몸 밑의 중앙부까지 이어지는 56에서 100개의 주름을 지니고 있는데, 먹이를 잡을 때 목을 팽창시키기 쉽게 하기 위한 것이다. 이들의 등지느러미의 길이는 60센티미터 정도이다. 가슴지느러미는 아주 작으며

In [ ]:
response = pipe(test_prompt, max_new_tokens=300, truncation=True)

print(response[0]['generated_text'])

<|im_start|>system
당신은 거만한 QA 전문 봇 입니다. 아래의 주어진 Context를 바탕으로 질문에 답변해주세요.<|im_end|>
<|im_start|>user
Context: 참고래는 전체적으로 몸통이 길고 날씬하므로 쉽게 구별할 수 있다. 암컷과 수컷의 평균 몸길이는 각각 19에서 20미터 정도이다. 북반구에 분포하는 아종은 24미터까지 자랄 수 있으며, 남극 지방의 아종은 최대 26.8미터나 된다.[7] 성체의 몸무게를 직접 계량한 적은 없지만, 추측치로 몸길이 25미터의 개체는 70톤이 나갈 수 있다는 결과가 있다. 완전히 성숙하는 데에는 상당한 기간이 필요한데 25년에서 30년이 걸린다.[18] 최대 94년까지 산 것이 확인된 적이 있다.[18] 갓 태어난 참고래의 몸길이는 6.5미터 정도이며, 몸무게는 1,800킬로그램 정도이다.[19] 이들의 어마어마한 크기는 그들을 다른 고래들과 구분하기에 충분하며, 때때로 대왕고래나 보리고래 같은 다른 대형 고래와 혼동될 뿐이다.

참고래의 등 부분은 밤회색이며, 배 쪽은 하얗다. 튀어나온 두 쌍의 숨구멍이 있으며, 납작하고 넓은 주둥이를 가지고 있다. 두 개의 밝은 색 문양이 숨구멍 뒤에서 시작해 몸의 측면으로 따라가 꼬리로 이어진다.[7] 오른쪽 턱에 하얀색 무늬가 있으며, 왼쪽은 회색 또는 검은색이다.[19] 이러한 비대칭성은 쇠정어리고래에게도 종종 발견되지만, 참고래들에게는 비대칭성이 진부하며, 다른 고래에게는 이러한 특성을 찾아볼 수 없으므로 다른 고래들과 구분하는 한 가지 척도가 되고 있다. 비대칭성은 그들이 돌 때 오른쪽으로 돌아 그렇게 됐다는 가설이 있지만, 실제로는 왼쪽으로도 돌기도 한다. 아직까지 이러한 비대칭성을 자세히 설명할 만한 가설은 없다.[20]

참고래는 턱에서 몸 밑의 중앙부까지 이어지는 56에서 100개의 주름을 지니고 있는데, 먹이를 잡을 때 목을 팽창시키기 쉽게 하기 위한 것이다. 이들의 등지느러미의 길이는 60센티미터 정도이다. 가슴지느러미는 아주 작으며

## PEFT(Prompt-Efficient Fine Tuning) 학습
전체 파인 튜닝을 하지 않고 PEFT를 사용하면 파라미터의 수를 줄인 효과적인 튜닝이 가능

In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [ ]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig,get_peft_model

In [ ]:
config = LoraConfig(
    r=8,                      # 랭크 값 (적은 수일수록 파라미터 감소)
    lora_alpha=32,            # LoRA scaling factor
    target_modules=[          # LoRA를 적용할 모듈 (보통 Transformer block의 Linear 계층)
        "q_proj", "k_proj", "v_proj", "o_proj",
        "up_proj", "down_proj", "gate_proj"
    ],
    lora_dropout=0.05,
    bias="none",              # bias 학습 여부
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

In [ ]:
# 학습되는 파라미터 수 출력
model.print_trainable_parameters()

trainable params: 9,232,384 || all params: 1,552,946,688 || trainable%: 0.5945


Huggingface의 trl 라이브러리는 파인 튜닝을 쉽게 수행하게 해 주는 라이브러리  
Supervised Fine Tuning - `SFTTrainer`를 불러와서 수행

In [ ]:
data['train']

Dataset({
    features: ['context', 'question', 'answer', 'text'],
    num_rows: 30
})

In [ ]:
from trl import SFTTrainer, SFTConfig

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

sft_config = SFTConfig(
    max_steps=400,
    dataset_text_field="text",                # 데이터셋 중 학습할 텍스트 필드명
    per_device_train_batch_size=1,            # GPU 1개당 배치 사이즈
    gradient_accumulation_steps=1,            # 그라디언트 누적 (메모리 부족할 때 유용)
    max_seq_length=1024,                      # 시퀀스 최대 길이
    lr_scheduler_type='cosine',               # learning rate 스케줄러
    learning_rate=1e-4,                       # 초기 학습률
    fp16=True,                                # float16 훈련
    optim="paged_adamw_8bit",                 # 8bit optimizer 사용 (메모리 절약)
    output_dir="outputs",                     # 체크포인트 저장 폴더
    logging_steps=25,                         # 손실 출력 빈도
    save_steps=50                             # 체크포인트 저장 빈도
)

trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=sft_config,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 25080 has 14.73 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 28.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

전체 text가 아닌 일부에 대해서만 학습
 - 데이터의 형식에 따라, 답변에 해당하는 부분만을 학습

```python
response_template = " ### Answer:"

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
...
data_collator=collator, # Completion에 대해서만 학습
)

```

# 학습 결과 확인

In [ ]:
test_prompt

In [ ]:
test_context = '''참고래는 전체적으로 몸통이 길고 날씬하므로 쉽게 구별할 수 있다. 암컷과 수컷의 평균 몸길이는 각각 19에서 20미터 정도이다. 북반구에 분포하는 아종은 24미터까지 자랄 수 있으며, 남극 지방의 아종은 최대 26.8미터나 된다.[7] 성체의 몸무게를 직접 계량한 적은 없지만, 추측치로 몸길이 25미터의 개체는 70톤이 나갈 수 있다는 결과가 있다. 완전히 성숙하는 데에는 상당한 기간이 필요한데 25년에서 30년이 걸린다.[18] 최대 94년까지 산 것이 확인된 적이 있다.[18] 갓 태어난 참고래의 몸길이는 6.5미터 정도이며, 몸무게는 1,800킬로그램 정도이다.[19] 이들의 어마어마한 크기는 그들을 다른 고래들과 구분하기에 충분하며, 때때로 대왕고래나 보리고래 같은 다른 대형 고래와 혼동될 뿐이다.

참고래의 등 부분은 밤회색이며, 배 쪽은 하얗다. 튀어나온 두 쌍의 숨구멍이 있으며, 납작하고 넓은 주둥이를 가지고 있다. 두 개의 밝은 색 문양이 숨구멍 뒤에서 시작해 몸의 측면으로 따라가 꼬리로 이어진다.[7] 오른쪽 턱에 하얀색 무늬가 있으며, 왼쪽은 회색 또는 검은색이다.[19] 이러한 비대칭성은 쇠정어리고래에게도 종종 발견되지만, 참고래들에게는 비대칭성이 진부하며, 다른 고래에게는 이러한 특성을 찾아볼 수 없으므로 다른 고래들과 구분하는 한 가지 척도가 되고 있다. 비대칭성은 그들이 돌 때 오른쪽으로 돌아 그렇게 됐다는 가설이 있지만, 실제로는 왼쪽으로도 돌기도 한다. 아직까지 이러한 비대칭성을 자세히 설명할 만한 가설은 없다.[20]

참고래는 턱에서 몸 밑의 중앙부까지 이어지는 56에서 100개의 주름을 지니고 있는데, 먹이를 잡을 때 목을 팽창시키기 쉽게 하기 위한 것이다. 이들의 등지느러미의 길이는 60센티미터 정도이다. 가슴지느러미는 아주 작으며, 꼬리는 넓고 V자 모양이며 끝은 뾰족한 편이다.[7]

이들이 수면에 떠오를 때 분출 후 곧 등지느러미가 보인다. 분출은 수직이며 가늘고 최대 높이는 6미터 정도이다.[19] 수면 위로 떠오를 때 여러 번 분출을 하며, 1분 30초 동안 머문다. 꼬리는 언제나 물속에 잠겨 있다. 그들의 잠수 깊이는 최대 250미터이며, 잠수 시간은 10 ~ 15분이다. 때때로 다른 고래들처럼 점프를 함으로써 몸 전체를 들어 올리고는 한다.'''

test_question = '''Question: 참고래는 무슨 색깔인가요?'''

test_prompt = convert_prompt(test_context,test_question)

In [ ]:
model.eval()

response = pipe(test_prompt, max_new_tokens=500, truncation=True, temperature=0.1)

print(response[0]['generated_text'])

# 모델 저장 및 huggingface 업로드

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

login(hf_token)

In [ ]:
model.save_pretrained('./Rude_RAG')

In [ ]:
model.push_to_hub('rude_rag')

In [ ]:
tokenizer.push_to_hub('rude_rage')

## huggingface 모델과 Langchain 연동

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    temperature=0.1,
    return_full_text=True
)

In [ ]:
!pip install langchain langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

In [ ]:
test_context = '''참고래는 전체적으로 몸통이 길고 날씬하므로 쉽게 구별할 수 있다. 암컷과 수컷의 평균 몸길이는 각각 19에서 20미터 정도이다. 북반구에 분포하는 아종은 24미터까지 자랄 수 있으며, 남극 지방의 아종은 최대 26.8미터나 된다.[7] 성체의 몸무게를 직접 계량한 적은 없지만, 추측치로 몸길이 25미터의 개체는 70톤이 나갈 수 있다는 결과가 있다. 완전히 성숙하는 데에는 상당한 기간이 필요한데 25년에서 30년이 걸린다.[18] 최대 94년까지 산 것이 확인된 적이 있다.[18] 갓 태어난 참고래의 몸길이는 6.5미터 정도이며, 몸무게는 1,800킬로그램 정도이다.[19] 이들의 어마어마한 크기는 그들을 다른 고래들과 구분하기에 충분하며, 때때로 대왕고래나 보리고래 같은 다른 대형 고래와 혼동될 뿐이다.

참고래의 등 부분은 밤회색이며, 배 쪽은 하얗다. 튀어나온 두 쌍의 숨구멍이 있으며, 납작하고 넓은 주둥이를 가지고 있다. 두 개의 밝은 색 문양이 숨구멍 뒤에서 시작해 몸의 측면으로 따라가 꼬리로 이어진다.[7] 오른쪽 턱에 하얀색 무늬가 있으며, 왼쪽은 회색 또는 검은색이다.[19] 이러한 비대칭성은 쇠정어리고래에게도 종종 발견되지만, 참고래들에게는 비대칭성이 진부하며, 다른 고래에게는 이러한 특성을 찾아볼 수 없으므로 다른 고래들과 구분하는 한 가지 척도가 되고 있다. 비대칭성은 그들이 돌 때 오른쪽으로 돌아 그렇게 됐다는 가설이 있지만, 실제로는 왼쪽으로도 돌기도 한다. 아직까지 이러한 비대칭성을 자세히 설명할 만한 가설은 없다.[20]

참고래는 턱에서 몸 밑의 중앙부까지 이어지는 56에서 100개의 주름을 지니고 있는데, 먹이를 잡을 때 목을 팽창시키기 쉽게 하기 위한 것이다. 이들의 등지느러미의 길이는 60센티미터 정도이다. 가슴지느러미는 아주 작으며, 꼬리는 넓고 V자 모양이며 끝은 뾰족한 편이다.[7]

이들이 수면에 떠오를 때 분출 후 곧 등지느러미가 보인다. 분출은 수직이며 가늘고 최대 높이는 6미터 정도이다.[19] 수면 위로 떠오를 때 여러 번 분출을 하며, 1분 30초 동안 머문다. 꼬리는 언제나 물속에 잠겨 있다. 그들의 잠수 깊이는 최대 250미터이며, 잠수 시간은 10 ~ 15분이다. 때때로 다른 고래들처럼 점프를 함으로써 몸 전체를 들어 올리고는 한다.'''

test_question = '''Question: 참고래는 먹이를 잡을 때 어떻게 하나요?'''

In [ ]:
chat = [
        {'role':'user','content':f'''Context: {test_context}

---

{test_question}'''}
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, temperature=0.1)

print(prompt)

In [ ]:
llm.invoke(prompt)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
[
        ('user','''Context: {context}

---

{question}''')
]
)

In [ ]:
test_context2 = '''임페라토르 카이사르 디비 필리우스 아우구스투스(Imperator Caesar divi filius Augustus, 기원전 63년 9월 23일 ~ 서기 14년 8월 19일)는 로마 제국 초대 황제(재위 기원전 27년 ~ 서기 14년)이다. 또한 로마 제국의 첫 번째 황조인 율리우스-클라우디우스 왕조의 초대 황제이기도 하다. 본명은 가이우스 옥타비우스 투리누스(Gaius Octavius Thurinus)였으나, 카이사르의 양자로 입적된 후 가이우스 율리우스 카이사르 옥타비아누스(Gaius Julius Caesar Octavianus)로 불렸다. 기원전 44년 옥타비아누스는 자신의 외할머니 율리아 카이사리스의 남동생이자 자신의 외종조부뻘인 율리우스 카이사르가 암살되자, 유언장에 따라 카이사르의 양자가 되어 그 후계자가 되었다. 기원전 43년, 옥타비아누스는 마르쿠스 안토니우스, 마르쿠스 아이밀리우스 레피두스와 함께 군사 정권인 제2차 삼두 정치를 열었다. 삼두 정치를 행한 집정관의 한 사람으로서 옥타비아누스는 효과적으로 로마와 속주[1]를 지배하였고, 세력을 모아 히르티우스와 판사가 사후 집정관에 재선되었다. 이후 제2차 삼두 정치도 깨지는데 다른 집정관이었던 레피두스는 유배되고 마르쿠스 안토니우스는 기원전 31년 악티움 해전에서 패배한 뒤 자살하였다.

제2차 삼두 정치의 붕괴 후 옥타비아누스는 대외적으로 로마 공화국을 부활시키고 정부에 관한 권한은 로마 원로원에게 주었으나, 사실상 권력을 독점하였다. 유일한 통치자가 다스리지만 대외적으로는 공화국 형태인 정치 체제의 기틀을 다지는 데는 오랜 시간이 걸렸다. 껍데기만 공화국인 이 나라는 훗날 로마 제국으로 불린다. 황제권은 옥타비아누스 이전에 로마를 통치했던 카이사르와 술라의 독재권과는 전혀 달랐다. 옥타비아누스는 로마의 원로원과 시민들로부터 “독재권을 부여받았지만” 거절하였다.[2] 법에 따르면 ‘존엄자’(아우구스투스)라는 칭호를 받은 옥타비아누스에게 원로원은 평생 동안 권력을 가지도록 하였고 “호민관 권한”(tribunitia potestas)을 가졌으며 기원전 23년까지 집정관을 역임하였다.[3] 아우구스투스는 재정적인 성공과 원정에서 얻은 물자, 제국 전체에 걸쳐 맺은 여러 피호 관계(clientela), 군인과 재향 군인의 충성, 원로원에서 부여한 여러 권한과 명예[4] 그리고 사람들의 존경을 받아 절대적인 권력을 누렸다. 아우구스투스가 가진 로마의 정예병 로마 군단 다수를 통제할 수 있는 권한은 원로원에게 군사적인 위협이 되어 원로원의 결정을 억압하였고, 군사적 수단을 사용하여 원로원의 정적들을 제거하여 원로원이 자신에 복종하게끔 하였다.'''

test_question2 = '''Question: 아우구스투스가 로마 제국을 통치한 기간은 얼마나 되나요?'''


qa_chain = chat_prompt | chat_model | StrOutputParser()
qa_chain.invoke({'question':test_question2, 'context':test_context2})